In [ ]:
base_dir = '/home/wkeeley/Desktop/Brain-Hemorrhaging'
image_dir = os.path.join(base_dir, 'Images')
train_dir = os.path.join(image_dir, 'Train_Images')
val_dir = os.path.join(image_dir, 'Validation_Images')

dcm_dir = os.path.join(base_dir, 'stage_1_train_images')
dcm_test_dir = os.path.join(base_dir, 'stage_1_test_images')

sub_train_dirs = []
sub_val_dirs = []

for i in sub_types:
    sub_train_dirs.append(os.path.join(train_dir, i))
    sub_val_dirs.append(os.path.join(val_dir, i))

In [62]:
#model One
from keras import layers
from keras import models
from keras import Input

input_tensor = Input(shape=(512, 512, 3))

branch_a = layers.Conv2D(512, 1, strides=2, activation='relu')(input_tensor)

branch_b = layers.SeparableConv2D(512, 1, activation='relu')(input_tensor)
branch_b = layers.SeparableConv2D(512, 2, padding='same', activation='relu')(branch_b)
branch_b = layers.SeparableConv2D(512, 4, padding='same', activation='relu')(branch_b)
branch_b = layers.SpatialDropout2D(0.2)(branch_b)
branch_b = layers.AveragePooling2D(2, strides=2)(branch_b)
branch_b = layers.SeparableConv2D(512, 2, padding='same', activation='relu')(branch_b)
branch_b = layers.SeparableConv2D(512, 2, padding='same', activation='relu')(branch_b)
branch_b = layers.SpatialDropout2D(0.2)(branch_b)
branch_b = layers.SeparableConv2D(512, 2, padding='same', activation='relu')(branch_b)
branch_b = layers.SeparableConv2D(512, 2, padding='same', activation='relu')(branch_b)


branch_c = layers.SeparableConv2D(512, 1, activation='relu')(input_tensor)
branch_c = layers.SeparableConv2D(512, 2, padding='same', activation='relu')(branch_c)
branch_c = layers.SeparableConv2D(512, 4, strides=2, padding='same', activation='relu')(branch_c)

branch_d = layers.SeparableConv2D(512, 1, activation='relu')(input_tensor)
branch_d = layers.SeparableConv2D(512, 1, activation='relu')(branch_d)
branch_d_1 = layers.SeparableConv2D(512, 1, activation='relu')(branch_d)
branch_d_1 = layers.SeparableConv2D(512, 1, activation='relu')(branch_d_1)
branch_d_1 = layers.BatchNormalization()(branch_d_1)
branch_d_1 = layers.SeparableConv2D(512, 4, padding='same', activation='relu')(branch_d_1)
branch_d_1 = layers.SeparableConv2D(512, 4, padding='same', activation='relu')(branch_d_1)
branch_d_1 = layers.add([branch_d_1, branch_d])
branch_d_1 = layers.SeparableConv2D(512, 1, strides=2, activation='relu')(branch_d_1)


out = layers.concatenate([branch_a, branch_b, branch_c, branch_d_1], axis=-1)
out = layers.Dense(1, activation='sigmoid')(out)
print(out.shape)

(?, 256, 256, 1)


In [3]:
#Intraparechymal data Gen.
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
def callbacks(category):
    log_dir = os.path.join(base_dir, category + '_logs')
    try:
        os.mkdir(log_dir)
    except FileExistsError:
        pass
    
    callbacks = [
    keras.callbacks.EarlyStopping(monitor='acc', patience=2),
    keras.callbacks.ModelCheckpoint(file_path=category + '.h5', 
                                    monitor='val_loss', 
                                    save_best_only=True),
    keras.callbacks.TensorBoard(log_dir=log_dir,
                               histogram_freq=1,
                               embeddings_freq=1)
    ]
    return callbacks
    
    

In [ ]:
#epidural
epidural_data = train_datagen.flow_from_dir(sub_train_dirs[0],
                                           target_size=(512, 512),
                                           batch_size=128,
                                           class_mode='binary')

epidural_val_data = test_datagen.flow_from_directory(sub_val_dirs[0],
                                                       target_size=(512, 512),
                                                       batch_size=128,
                                                       class_mode='binary')
epidural_callbacks = callbacks('epidural')

In [ ]:


out.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
history = out.fit_generator(epidural_data,
                           steps_per_epoch=100,
                           epochs=30,
                           callbacks=epidural_callbacks,
                           validation_data=epidural_val_data,
                           validation_steps=50)

In [ ]:
intraparenchymal_data = train_datagen.flow_from_dir(sub_train_dirs[1],
                                           target_size=(512, 512),
                                           batch_size=128,
                                           class_mode='binary')

intraparenchymal_val_data = test_datagen.flow_from_directory(sub_val_dirs[1],
                                                       target_size=(512, 512),
                                                       batch_size=128,
                                                       class_mode='binary')

In [ ]:
intraparenchymal_callbacks = callbacks('intraparenchymal')
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
history = out.fit_generator(intraparenchymal_data,
                           steps_per_epoch=100,
                           epochs=30,
                           callbacks=intraparenchymal_callbacks,
                           validation_data=intraparenchymal_val_data,
                           validation_steps=50)